In [1]:
import spacy
import pandas as pd
import numpy as np

2023-02-13 20:35:06.248589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 20:35:06.429633: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-13 20:35:06.465179: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-13 20:35:08.679777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
df = pd.read_csv('expert_data.tsv',delimiter = '\t', on_bad_lines='skip')

df.head()

/scratch/ials-gpu025/5323942/ipykernel_2031976/2456222905.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('expert_data.tsv',delimiter = '\t', on_bad_lines='skip')


,id,text_to_annotate,start,end,ann_text,definition
0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi..."
1,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,7.0,8,1,{DO NOT DEFINE}
2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9,MG,"A tiny amount of something, usually a drug."
3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10,Oral,Taken by mouth.
4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11,Tablet,A pill.


In [3]:
import scispacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [4]:
import en_core_sci_lg
model = spacy.load('en_core_sci_lg')
model.add_pipe("scispacy_linker",config = {"resolve_abbreviations":True,"linker_name":"umls"}) #here in place of umls we can also use mesh, rxnorm, go, hpo
  
"""
attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, 
token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, 
entity_linker, ner, beam_ner, entity_ruler, tagger, morphologizer, senter, sentencizer, 
textcat, spancat, future_entity_ruler, span_ruler, textcat_multilabel, scispacy_linker, en.lemmatizer

"""

/work/pi_hongyu_umass_edu/zonghai/nandyala/Anaconda/envs/UMLS/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/work/pi_hongyu_umass_edu/zonghai/nandyala/Anaconda/envs/UMLS/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


'\nattribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, \ntoken_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, \nentity_linker, ner, beam_ner, entity_ruler, tagger, morphologizer, senter, sentencizer, \ntextcat, spancat, future_entity_ruler, span_ruler, textcat_multilabel, scispacy_linker, en.lemmatizer\n\n'

In [5]:
linker = model.get_pipe("scispacy_linker")

In [6]:
doc = model("protonix") 

In [7]:
entity = doc.ents[0]
print("Name: ", entity)

Name:  protonix


In [8]:
entity._.kb_ents[0]


('C0876139', 1.0)

In [9]:
for umls_ent in entity._.kb_ents:
    print(linker.kb.cui_to_entity[umls_ent[0]]) #number 4 is definition

CUI: C0876139, Name: Protonix
Definition: None
TUI(s): T109, T121
Aliases: (total: 2): 
	 protonix, Protonix


In [11]:
import torch
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [12]:
jterm = "Spinal and bulbar muscular atrophy (SBMA) is an inherited motor neuron disease caused by the expansion of a polyglutamine tract within the androgen receptor (AR). SBMA can be caused by this easily."

lay_def = "Spinal and bulbar muscular atrophy mainly affects males and is characterized by muscle weakness and wasting (atrophy) that usually begins in adulthood and worsens slowly over time"

In [13]:
jj = "inguinal"
ll = "To do with the area in between the fold of the groin"

In [14]:
jj1 = "periventricular"
ll1 = "Abnormal low attenuation is noted in the subcortical and periventricular deep white matter of both cerebral hemispheres suggestive of small vessel chronic ischemic disease."

In [15]:
wl = ["Spinal", "bulbar muscular atrophy", "SBMA", "inherited", "motor neuron disease", "expansion", "polyglutamine tract", "androgen receptor", "AR", "SBMA"]

In [16]:
def sentence_split(sentence, word_list):
    split = []
    for w in word_list:
        begin = sentence[:sentence.index(w)]
        if begin != "" and begin != " ":
            split.append(begin)
        split.append(w)
        sentence = sentence[sentence.index(w)+len(w):]
    if sentence != "":
        split.append(sentence)
#     print(split)
    return split


#     begin = sentence[:sentence.index(word_list[0])]
#     print(begin)
#     end = sentence[sentence.index(word_list[1])+len(word_list[0]):]
    
#     print(end)

In [17]:
sentence_split(jterm,wl)

['Spinal',
 ' and ',
 'bulbar muscular atrophy',
 ' (',
 'SBMA',
 ') is an ',
 'inherited',
 'motor neuron disease',
 ' caused by the ',
 'expansion',
 ' of a ',
 'polyglutamine tract',
 ' within the ',
 'androgen receptor',
 ' (',
 'AR',
 '). ',
 'SBMA',
 ' can be caused by this easily.']

In [24]:
# forgot to put try as corpus could be empty
def case4(jterm,lay_def):
#     print(jterm)
#     jterm = str(jterm)
#     lay_def = str(lay_def)
    docc = model(jterm)
    arr = []
    for i in range(len(docc.ents)):
        entt = docc.ents[i]
        arr.append(entt)
    gen_def = []
    Dict = {}
#     print("arr is ",arr)
#     print(type(arr[0]))
    for jt in arr:
        corpus = []
        for umls_ent in jt._.kb_ents:
            corpus.append(linker.kb.cui_to_entity[umls_ent[0]][4])
        if len(corpus) == 0 or corpus[0] == None:
            corpus.append(jt.text)
        corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
        lay_def_emb = embedder.encode(lay_def, convert_to_tensor=True)
        cos_scores = util.cos_sim(lay_def_emb, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=1)
#         print(corpus)
        for score, idx in zip(top_results[0], top_results[1]):
            gen_def.append(corpus[idx])
            Dict[jt.text] = corpus[idx]
    arr = [i.text for i in arr]
    sent_split = sentence_split(jterm, arr)
    copyy = sent_split.copy()
#     print(sent_split)
    for i in range(len(sent_split)):
        if sent_split[i] in Dict:
            sent_split[i] = Dict[sent_split[i]]
            
#     print(sent_split)
    return sent_split,copyy

In [19]:
case4(jterm,lay_def)
# jterm = "Spinal and bulbar muscular atrophy (SBMA) is an 
# inherited motor neuron disease caused by the expansion of a polyglutamine 
# tract within the androgen receptor (AR). SBMA can be caused by this easily."

(['Of or relating to the spine or spinal cord.',
  ' and ',
  'An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.',
  ' (',
  'An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.',
  ') is an ',
  'A hereditary disorder that affects the sensory and/or motor nerves or the autonomic nerves.',
  'Diseases characterized by a selective degeneration of the motor neurons of the spinal cord, brainstem, or motor cortex. Clinical subtypes are distinguished by the major site of degeneration. In AMYOTROPHIC LATERAL SCLEROSIS there is involvement of upper, lower, and brainstem motor neurons. In progressive muscular atrophy and related syndromes (see MUSCULAR ATROPHY, SPINAL) the motor neurons in the spinal cord are primarily affected. With progressive bulbar palsy (BULBAR PALSY, PROGRESSIVE), the initial degeneration occurs in the brainstem. In primary later

In [46]:
case4(jj,ll)

(['The external junctural region between the lower part of the abdomen and the thigh.'],
 ['inguinal'])

In [47]:
case4(jj1,ll1)

(['A disorder resulting from a defect in the pattern of neuronal migration in which ectopic collections of neurons lie along the lateral ventricles of the brain or just beneath, contiguously or in isolated patches.'],
 ['periventricular'])

In [48]:
case4("abnormalities","Something that is not normal.")

(['disorders of the dental system that exist at, and usually before, birth regardless of their causation.'],
 ['abnormalities'])

In [49]:
case4("oozing","Fluid that leaks out of blood vessels.")

(['The material exuded from a wound, either surgical or traumatic.'],
 ['oozing'])

In [50]:
case4("Neupogen","The condition of remaining in a place or position or of holding within.")

(['A regimen consisting of filgrastim that may be used in the prevention of febrile neutropenia; in the harvest of peripheral blood stem cells (PBSCs); and in the symptomatic management of neutropenic disorders.'],
 ['Neupogen'])

In [51]:
case4("duodenal bulb","The part of the duodenum closest to the stomach.")

(['The shortest and widest portion of the SMALL INTESTINE adjacent to the PYLORUS of the STOMACH. It is named for having the length equal to about the width of 12 fingers.'],
 ['duodenal bulb'])

In [20]:
case4("Virt - Vite","A mix of vitamins. It provides vitamin B-6")

(['Virt',
  ' - ',
  'The determination of the amount of Vitamin E present in a sample.'],
 ['Virt', ' - ', 'Vite'])

In [23]:
case4("protonix","A drug used to treat heartburn and ulcers. It lowers the amount of acid the stomach makes.")

(['protonix'], ['protonix'])

In [26]:
print("lets use the above functions to create a case 3 and case 4 data")

lets use the above functions to create a case 3 and case 4 data


In [25]:
df = pd.read_csv('expert_data.tsv',delimiter = '\t', on_bad_lines='skip')

df.head()

/scratch/ials-gpu025/5323942/ipykernel_2031976/2456222905.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('expert_data.tsv',delimiter = '\t', on_bad_lines='skip')


,id,text_to_annotate,start,end,ann_text,definition
0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi..."
1,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,7.0,8,1,{DO NOT DEFINE}
2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9,MG,"A tiny amount of something, usually a drug."
3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10,Oral,Taken by mouth.
4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11,Tablet,A pill.


In [26]:
df["split_print"] = ""
df["UMLS+others"] = ""
df["UMLS"] = ""

In [27]:
# df["split_print"],df["UMLS+others"] = df.apply(lambda x : case4(x.ann_text,x.definition),axis = 1)

In [28]:
for i in range(len(df)):
    try:
        df.at[i,"split_print"],df.at[i,"UMLS+others"] = case4(df.iloc[i]["ann_text"],df.iloc[i]["definition"])
    except:
        print(df.iloc[i]["ann_text"])  
        print("def is")
        print(df.iloc[i]["definition"])

nan
def is
nan
nan
def is
An element in the blood. Too much or too little can cause major problems.
nan
def is
nan
nan
def is
An element in the blood. Too much or too little can cause major problems.
Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
	Tissue that is thick and scarred, usually from an injury.
def is
nan
nan
def is
nan
nan
def is
nan
	Muscles that assist breathing.
def is
nan
	Having to do with the thigh bone.
def is
nan
nan
def is
An element in the blood. Too much or too little can cause major problems.
nan
def is
nan
nan
def is
nan


In [29]:
df.to_csv("case4.csv")

In [30]:
df.to_pickle("case4.pkl")